## Import

In [1]:
import os
from radiomics import featureextractor
import numpy as np
import nrrd
import SimpleITK as sitk
import csv
from radiomics import getFeatureClasses
import pandas as pd
import zipfile

import logging
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)


## Functions

In [ ]:

# Load volume or mask
def carica_volume(file_path):
    return nrrd.read(file_path)[0]

# Function to extract the features
def estrai_features_slice(slice_immagine, slice_maschera, estrattore, idx):
    if not np.any(slice_maschera > 0):
        return None, None
    
    try:
        sitk_slice_immagine = numpy_to_sitk(slice_immagine)
        sitk_slice_maschera = numpy_to_sitk(slice_maschera)
        results = estrattore.execute(sitk_slice_immagine, sitk_slice_maschera)
        # Transform the results
        feature_vector = np.array([value.item() if isinstance(value, np.ndarray) else value 
                                   for key, value in results.items() 
                                   if isinstance(value, (float, int, np.ndarray))])
        return feature_vector, results
    except Exception as e:
        print(f"Errore slice {idx}: {e}")  
        return None, None

#Function to find the largest plaque
def get_largest_slice_index(image_array, mask_array):
    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array)
    
    slice_sums = np.sum(mask_array[zstart-1:zstop+1, ystart:ystop, xstart:xstop], axis=(1, 2))
    largest_slice_index = np.argmax(slice_sums)
    
    return largest_slice_index

# Extract the bounding box of the mask
def maskcroppingbox(images_array):
    images_array_2 = np.argwhere(images_array)
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)

# Create 2.5D csv with features names
def crea_csv_features_25D(pazienti_dati, features_name, csv_file):
    if not pazienti_dati:
        return

    headers = ['Paziente', 'Slice'] + features_name

    with open(csv_file, mode='w', newline='') as file_csv:
        writer = csv.writer(file_csv)
        writer.writerow(headers)

        for paziente, slice_idx, feature_vector in pazienti_dati:
            writer.writerow([paziente, slice_idx] + list(feature_vector))

# Convert from numpy to sitk
def numpy_to_sitk(array_numpy):
    array_numpy = np.expand_dims(array_numpy, axis = -1)
    return sitk.GetImageFromArray(array_numpy)


# Find nrrd file in a folder
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None


## 2.5D

In [ ]:
print(list(getFeatureClasses().keys()))

# Folders
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'


pazienti_dati = []
# Define the extractor
estrattore = featureextractor.RadiomicsFeatureExtractor(force2D=True, force2Ddimension=2)

# Define the features to extract
estrattore.disableAllFeatures()
estrattore.enableFeatureClassByName('firstorder')
estrattore.enableFeatureClassByName('glcm')
estrattore.enableFeatureClassByName('gldm')
estrattore.enableFeatureClassByName('glrlm')
estrattore.enableFeatureClassByName('glszm')
estrattore.enableFeatureClassByName('ngtdm')
estrattore.enableImageTypeByName('Wavelet')
estrattore.enableFeatureClassByName('shape2D')

#print("Feature Types Abilitati:", estrattore.enabledImagetypes)

for paziente_folder in sorted(os.listdir(pazienti_dir)):
    paziente_path = os.path.join(pazienti_dir, paziente_folder)
    roi_path = os.path.join(roi_dir, paziente_folder)

    print(paziente_folder)

    if os.path.isdir(paziente_path) and os.path.isdir(roi_path):
        file_immagine_nrrd = trova_nrrd_file(paziente_path)
        file_maschera_nrrd = trova_nrrd_file(roi_path)
        
        if file_immagine_nrrd and file_maschera_nrrd:
            volume_immagine = carica_volume(file_immagine_nrrd)
            volume_maschera = carica_volume(file_maschera_nrrd)

            if volume_immagine.shape != volume_maschera.shape:
                print(f"Il volume dell'immagine e la maschera non hanno la stessa forma per il paziente {paziente_folder}")
                continue
            
            # Iterate over every slice
            for idx in range(volume_immagine.shape[2]):
                slice_immagine = volume_immagine[:, :, idx]
                slice_maschera = volume_maschera[:, :, idx]

                if np.any(slice_maschera):
                    feature_vector, results = estrai_features_slice(slice_immagine, slice_maschera, estrattore, idx)
                else:
                    #print(f"Saltato {idx}")
                    continue
                if feature_vector is not None:
                    features_name = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]
                    pazienti_dati.append((paziente_folder, idx + 1, feature_vector))

# Order by patient and slice
pazienti_dati.sort(key=lambda x: (int(x[0]), x[1]))

# Create csv to the specified path
#crea_csv_features_25D(pazienti_dati, features_name, 'C:\\Users\\bsbar\\Desktop\\Radiomica_Wavelet_25D.csv')

## 2D

In [ ]:
# Folders
pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"


largest_slices_per_patient = []
# Iterate through the ordered patients
for s in sorted(os.listdir(pathdicom), key=int):
    #print(f"Processing patient: {s}")
    pathdicomnew = os.path.join(pathdicom, s)
    
    for t in os.listdir(pathdicomnew):
        dicom_path = os.path.join(pathdicomnew, t)
        readdatadicom, _ = nrrd.read(dicom_path, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        roi_path = os.path.join(pathroinew, g)
        readdatanrrd, _ = nrrd.read(roi_path, index_order='C')

    largest_slice_index = get_largest_slice_index(readdatadicom, readdatanrrd)
    largest_slices_per_patient.append((s, largest_slice_index))


selected_slices = []

zip_file_path = '../CSVFeatures/2.5D/Radiomica_Wavelet_25D.csv.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    csv_filename = zip_ref.namelist()[0] 
    with zip_ref.open(csv_filename) as csv_file:
        df = pd.read_csv(csv_file)

for patient, optimal_slice_index in largest_slices_per_patient:
    
    patient = int(patient)
    patient_slices = df[df['Paziente'] == patient]
    print(optimal_slice_index)
    slice_number = patient_slices.iloc[optimal_slice_index]['Slice']
    optimal_slice_row = patient_slices[patient_slices['Slice'] == slice_number]
    selected_slices.append(optimal_slice_row)


#final_df = pd.concat(selected_slices)
#output_csv = 'C:\\Users\\bsbar\\Desktop\\Radiomica_Wavelet_2D.csv'
#final_df.to_csv(output_csv, index=False)